In [2]:
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.neighbors import LocalOutlierFactor
from sklearn.base import TransformerMixin
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler
from sklearn.preprocessing import OneHotEncoder
from catboost import CatBoostRegressor
import pandas as pd
import numpy as np


In [32]:
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [34]:
x = iris.drop(columns=['species'])
y = iris['species']

In [63]:
class DFSimpleImputer(TransformerMixin):
    def fit_transform(self,x,y):
        print('fit/trans')
        self.fit(x,y)
        return self.transform(x)
    def transform(self, x):
        print('trans')
        print(x.shape)
        return x
    def fit(self,x,y):
        print('fit')
        return x

In [64]:
from sklearn.ensemble import RandomForestClassifier
pipe = Pipeline(steps=[
    ('impute_num', DFSimpleImputer()),
    ('model',RandomForestClassifier())
    ])

In [ ]:
from sklearn import set_config
set_config(display='diagram')   
# diplays HTML representation in a jupyter context
column_trans  

In [65]:
pipe.fit(x,y)

fit/trans
fit
trans
(150, 4)


Pipeline(memory=None,
         steps=[('impute_num',
                 <__main__.DFSimpleImputer object at 0x000001E3D4769860>),
                ('model',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=False))],
         verbose=False)

In [66]:
from sklearn.model_selection import cross_validate
cross_validate(pipe, x,y,cv=3)

fit/trans
fit
trans
(100, 4)
trans
(50, 4)
fit/trans
fit
trans
(100, 4)
trans
(50, 4)
fit/trans
fit
trans
(100, 4)
trans
(50, 4)


{'fit_time': array([0.46672773, 0.59866142, 0.60865641]),
 'score_time': array([0.04297948, 0.03597403, 0.03497887]),
 'test_score': array([0.98, 0.94, 0.98])}

In [42]:
pipe.fit(x,y)

fit/trans


Pipeline(memory=None,
         steps=[('impute_num',
                 <__main__.DFSimpleImputer object at 0x000001E3D36A1D68>),
                ('model',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=False))],
         verbose=False)

In [1]:
import pandas as pd

In [140]:
import numpy as np
df= pd.DataFrame({'col1':[1,np.nan,3,4,5,np.nan],'Neighborhood':['bla','bla','niets','niets','test','test']})
for name, df in df.groupby('Neighborhood',as_index=False):
    print(df)

col1 Neighborhood
0   1.0          bla
1   NaN          bla
   col1 Neighborhood
2   3.0        niets
3   4.0        niets
   col1 Neighborhood
4   5.0         test
5   NaN         test


In [141]:
df.fillna(value={'2','3'})

TypeError: "value" parameter must be a scalar or dict, but you passed a "list"

In [176]:
from sklearn.base import TransformerMixin,BaseEstimator
class DFSmartImputer(TransformerMixin):
    def fit(self,X, y=None):
        self.conversion = {name: SimpleImputer(strategy='most_frequent').fit(df) for name, df in X.groupby('Neighborhood')}
        return self
    
    def transform(self, X,y=None):
        self.trans = [df.fillna(value=dict(zip(X.columns,self.conversion[name].statistics_))) for name, df in X.groupby('Neighborhood')]
        return pd.concat(self.trans)

In [179]:
imp = DFSmartImputer()
imp.fit(train_x)
a=imp.transform(train_x)

In [178]:
a.shape

(1456, 177)

ValueError: invalid fill value with a <class 'set'>

In [139]:
list(imp.conversion['MeadowV'].statistics_)

[160,
 'RM',
 21.0,
 1533,
 2,
 nan,
 4,
 'Lvl',
 'Inside',
 3,
 'MeadowV',
 'Norm',
 'Norm',
 'TwnhsE',
 '2Story',
 4,
 6,
 1970,
 1970,
 'Gable',
 'CompShg',
 'CemntBd',
 'CmentBd',
 'None',
 0.0,
 3,
 3,
 'CBlock',
 4.0,
 3.0,
 0.0,
 6.0,
 0,
 1.0,
 0,
 0,
 630,
 'GasA',
 3,
 'Y',
 'SBrkr',
 630,
 0,
 0,
 630,
 1,
 0,
 1,
 0,
 3,
 1,
 3,
 6,
 8,
 0,
 3.0,
 'Attchd',
 1970.0,
 'Unf',
 1,
 0,
 3.0,
 3.0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 nan,
 'MnPrv',
 nan,
 0,
 5,
 2009,
 'WD',
 'Normal',
 2,
 2,
 nan,
 1.0,
 1.0,
 1.0,
 4,
 1,
 1,
 2.0,
 1.0,
 1.0,
 2.0,
 1,
 1,
 24,
 9.0,
 9,
 3,
 3.0,
 858.0,
 nan,
 4,
 1,
 nan,
 286.0,
 1.0,
 1,
 2.0,
 1260,
 630,
 0,
 0.0,
 0,
 0,
 1,
 1,
 1,
 0,
 16,
 64,
 2.0,
 1587600,
 2000376000,
 35.4964786985977,
 396900,
 250047000,
 25.099800796022265,
 396900,
 250047000,
 25.099800796022265,
 4,
 8,
 1.4142135623730951,
 9,
 27,
 1.7320508075688772,
 1,
 1,
 1.0,
 4.0,
 8.0,
 1.4142135623730951,
 9,
 27,
 1.7320508075688772,
 736164.0,
 631628712.0,
 29.2